## 1. Import libraries, load data, clean data

In [14]:
# Import libraries

import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
import pandas as pd
import re

In [15]:
# Read in data

adjacency_matrix = pd.read_csv("adjacency_matrix2.csv", header = 0, index_col = 0)

multilevel = pd.read_csv("multilevel2.csv", header = 0, index_col = 0) 

In [16]:
# Clean the adjacency matrix

def clean_adjacency_mat(adjacency_matrix):
    # Reverse the order of columns 
    columns = adjacency_matrix.columns.tolist()
    columns = columns[::-1]
    adjacency_matrix = adjacency_matrix[columns]
    
    # Reverse the order of rows
    adjacency_matrix = adjacency_matrix[::-1]
    
    # Take the transpose of the matrix
    adjacency_matrix = adjacency_matrix.T
    
    return adjacency_matrix

adjacency_matrix = clean_adjacency_mat(adjacency_matrix)
adjacency_matrix

,Everything,CSF_508_5,Myelencephalon_507_5,Metencephalon_506_5,Mesencephalon_505_5,Diencephalon_R_504_5,Diencephalon_L_503_5,Telencephalon_R_502_5,Telencephalon_L_501_5,Sulcus_R_500_4,...,MFG_DPFC_R_10_1,MFG_DPFC_L_9_1,MFG_R_8_1,MFG_L_7_1,SFG_pole_R_6_1,SFG_pole_L_5_1,SFG_PFC_R_4_1,SFG_PFC_L_3_1,SFG_R_2_1,SFG_L_1_1
Everything,0,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
CSF_508_5,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
Myelencephalon_507_5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Metencephalon_506_5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Mesencephalon_505_5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SFG_pole_L_5_1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
SFG_PFC_R_4_1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
SFG_PFC_L_3_1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
SFG_R_2_1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
# Clean the multilevel lookup table

def clean_multilevel(multilevel):
    # Reverse the order of rows
    multilevel = multilevel[::-1]
    
    # Reassign the numbers
    multilevel.Number = range(1, adjacency_matrix.shape[0] + 1) 
    
    return multilevel

multilevel = clean_multilevel(multilevel)
multilevel

,Number,Structure,Immediate.parent,Immediate.child.children
509,1,Everything,NaN,"Telencephalon_L_501_5, Telencephalon_R_502_5, ..."
508,2,CSF_508_5,Everything,"Ventricle_498_4, Sulcus_L_499_4, Sulcus_R_500_4"
507,3,Myelencephalon_507_5,Everything,"Myelencephalon_L_494_4, Myelencephalon_R_495_4"
506,4,Metencephalon_506_5,Everything,"Metencephalon_L_492_4, Metencephalon_R_493_4"
505,5,Mesencephalon_505_5,Everything,"Mesencephalon_L_490_4, Mesencephalon_R_491_4"
...,...,...,...,...
5,505,SFG_pole_L_5_1,SFG_L_290_2,NaN
4,506,SFG_PFC_R_4_1,SFG_R_291_2,NaN
3,507,SFG_PFC_L_3_1,SFG_L_290_2,NaN
2,508,SFG_R_2_1,SFG_R_291_2,NaN


## 2. Create a subset of the adjacency matrix

In [182]:
def subset_matrix_creator(subset_leaf_list):
    full_subset_list = subset_leaf_list # This list will get filled
    iterations = 5 * len(subset_leaf_list) - 1
    
    for i in range(0, len(subset_leaf_list)): # For each leaf structure
        for j in range(0, iterations): # Iterate over the 4 other levels in this ontology (5 levels * number of leaves)

            # Structure index
            structure_index = int(np.where(multilevel["Structure"] == full_subset_list[j])[0])
            structure_index = (adjacency_matrix.shape[0]) - structure_index
            structure = multilevel["Structure"][structure_index]

            full_subset_list.append(multilevel["Immediate.parent"][structure_index])
        
    full_subset_list = set(full_subset_list)
    full_subset_list = pd.DataFrame(full_subset_list)
        
    pattern = r"_[0-9]+_"
    structure_numbers = []
    
    for i in range(0, full_subset_list.shape[0]):
        if full_subset_list[0][i] == "Everything":
            structure_numbers.append(float("inf")) # Assign the number "inf" to "Everything" for flexibility
        else:
            number = re.findall(pattern, full_subset_list[0][i])[0]
            number = re.sub("[^0-9]", "", number)
            number = int(number)
            structure_numbers.append(number)
    
    full_subset_list["structure_numbers"] = structure_numbers
    full_subset_list = full_subset_list.sort_values(by = ["structure_numbers"], axis = 0)
    full_subset_list = full_subset_list[0].tolist()
    
    # Index the rows and columns of the adjacency matrix by these structures to create a subset
    subset = adjacency_matrix[full_subset_list]
    subset = subset.loc[full_subset_list]

    # Reverse the order of rows and columns in the subset of the adjacency matrix
    cols = subset.columns.tolist()
    cols = cols[::-1]
    subset = subset[cols]
    subset = subset[::-1]
    
    return subset


# User inputs a list of leaf structures
subset_leaf_list = ["Amyg_L_73_1", "Hippo_L_75_1"]

subset = subset_matrix_creator(subset_leaf_list)
subset

,Everything,Telencephalon_L_501_5,CerebralCortex_L_482_4,Limbic_L_434_3,Hippo_L_338_2,Amyg_L_336_2,Hippo_L_75_1,Amyg_L_73_1
Everything,0,1,0,0,0,0,0,0
Telencephalon_L_501_5,0,0,1,0,0,0,0,0
CerebralCortex_L_482_4,0,0,0,1,0,0,0,0
Limbic_L_434_3,0,0,0,0,1,1,0,0
Hippo_L_338_2,0,0,0,0,0,0,1,0
Amyg_L_336_2,0,0,0,0,0,0,0,1
Hippo_L_75_1,0,0,0,0,0,0,0,0
Amyg_L_73_1,0,0,0,0,0,0,0,0


## 3. Creating adjacency matrices of descendants and ancestors for the subset

In [185]:
# Create an adjacency matrix of descendants

# N is the number of samples
# mu is the difference in mean, note that this is generally unknown
# M is the number of total unique structures

def adjacency_descendants(adjacency_matrix, N, mu, M):
    names_full = adjacency_matrix.columns # List of the 509 structures' names
    A = np.array(adjacency_matrix, dtype = bool)
    Descendants = np.copy(A)
    
    Descendants = np.logical_or(Descendants,Descendants@A)
    Descendants = np.logical_or(Descendants,Descendants@A)
    Descendants = np.logical_or(Descendants,Descendants@A)
    Descendants = np.logical_or(Descendants,Descendants@A)
    Descendants = np.logical_or(Descendants,Descendants@A)
    Descendants = np.logical_or(Descendants,Descendants@A)
    Descendants_and_self = np.logical_or(Descendants, np.eye(M))
    
    return Descendants

descendants = adjacency_descendants(subset, N = 20, mu = 3.0, M = subset.shape[0])
descendants

array([[False,  True,  True,  True,  True,  True,  True,  True],
       [False, False,  True,  True,  True,  True,  True,  True],
       [False, False, False,  True,  True,  True,  True,  True],
       [False, False, False, False,  True,  True,  True,  True],
       [False, False, False, False, False, False,  True, False],
       [False, False, False, False, False, False, False,  True],
       [False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False]])

In [188]:
# Create an adjacency matrix of ancestors

# N is the number of samples
# mu is the difference in mean, note that this is generally unknown
# M is the number of total unique structures

def adjacency_ancestors(adjacency_matrix, N, mu, M):
    Ancestors = descendants.T # Take transpose of descendants matrix to get ancestors    
    Ancestors_and_self = np.logical_or(Ancestors,np.eye(M))
    
    return Ancestors

ancestors = adjacency_ancestors(subset, N = 20, mu = 3.0, M = subset.shape[0])
ancestors

array([[False, False, False, False, False, False, False, False],
       [ True, False, False, False, False, False, False, False],
       [ True,  True, False, False, False, False, False, False],
       [ True,  True,  True, False, False, False, False, False],
       [ True,  True,  True,  True, False, False, False, False],
       [ True,  True,  True,  True, False, False, False, False],
       [ True,  True,  True,  True,  True, False, False, False],
       [ True,  True,  True,  True, False,  True, False, False]])

## 4. Professor Tward's functions for parameter estimation

In [189]:
# Define the function for phi (CDF of the standard normal distribution)
def phi(x,mu=0.0):
    '''Gaussian'''
    return 1.0/np.sqrt(2.0*np.pi)*np.exp(-(x - mu)**2/2.0)

In [190]:
# Define the function for calculating P (probabilities of Z) from Q (conditional probability of Z given parent)
def P_from_Q(Q,Ancestors_and_self):
    '''I don't need this function
    '''
    P = np.empty_like(Q)
    for i in range(M):
        P[i] = np.prod(Q[Ancestors_and_self[i,:]])
    return P

In [191]:
# Define the function for calculating Q (conditional probability of Z given parent) from P (probabilities of Z)
def Q_from_P(P,A):
    # now we need to calculate Q
    Q = np.zeros_like(P)
    Q[0] = P[0]
    for i in range(1,M):
        Q[i] = P[i] / P[A[:,i]]
    return Q

In [192]:
# Define the function for estimating P
def estimate_P(X,mu,A,Descendants_and_self,draw=False,niter=100,P0=None,names=None,clip=0.001):
    
    if draw: 
        f,ax = plt.subplots(2,2)
        if names is None:
            names = np.arange(A.shape[0])

    N = X.shape[0]
    m = X.shape[1]
    M = A.shape[0]
    
    # okay now comes my algorithm
    # initialize
    if P0 is None:
        P = np.ones(M)*0.5
    else:
        P = np.asarray(P0)
    
    for it in range(niter):
        # calculate leaf posterior (this is prob of no effect)
        #leaf_posterior = ((1.0-P[is_leaf])*phi(X))
        #leaf_posterior = leaf_posterior/(leaf_posterior + P[is_leaf]*phi(X,mu) )
        P_ = np.maximum(P, clip) # Clip probability: if P is very small, then set it to 0.001
        P_ = np.minimum(P_, 1-clip) # Clip probability: if P_ is very big, then set it to 0.999
        P_over_one_minus_P = P_/(1.0-P_)
        #leaf_log_posterior = -np.log(1.0 + P_over_one_minus_P[is_leaf]*phi(X,mu)/phi(X) )
        leaf_log_posterior = -np.log1p( P_over_one_minus_P[is_leaf]*phi(X,mu)/phi(X) )
        

        # calculate posterior for all structures
        # now for each structure, I need a leaf likelihod, and an adjustment
        #posterior = np.zeros((N,M))
        log_posterior = np.zeros((N,M))
        for i in range(M):
            #posterior[:,i] = np.prod(leaf_posterior[:,Descendants_and_self[i,:][is_leaf]],1)
            log_posterior[:,i] = np.sum(leaf_log_posterior[:,Descendants_and_self[i,:][is_leaf]],1)
        
        # calculate adjustment factor for correlations
        Q = Q_from_P(P,A)
        #adjustment_single = np.ones(M)
        log_adjustment_single = np.zeros(M)
        for i in range(M):
            if is_leaf[i]:
                continue
            #adjustment_single[i] = (1.0 - P[i])/ ((1.0 - P[i]) + P[i]*np.prod(1.0 - Q[A[i,:]]))
            #log_adjustment_single[i] = -np.log(1.0 + P_over_one_minus_P[i]*np.prod(1.0 - Q[A[i,:]]))
            log_adjustment_single[i] = -np.log1p(P_over_one_minus_P[i]*np.prod(1.0 - Q[A[i,:]]))
            
        
        # now my adjust ment requres products of all descendants
        #adjustment = np.ones(M)
        log_adjustment = np.ones(M)
        for i in range(M):
            #adjustment[i] = np.prod(adjustment_single[Descendants_and_self[i,:]])
            log_adjustment[i] = np.sum(log_adjustment_single[Descendants_and_self[i,:]])
            

        # calculate the adjusted posterior
        #posterior = posterior*adjustment
        log_posterior = log_posterior + log_adjustment
        
        #P = np.sum(1.0 - posterior,0)/N        
        #P = np.sum(1.0 - np.exp(log_posterior),0)/N
        P = -np.sum(np.expm1(log_posterior),0)/N
        posterior = np.exp(log_posterior)
        
        # draw        
        if draw>0 and ( (not it%draw) or (it==niter-1)):     
            
            ax[0,0].cla()
            ax[0,0].imshow(posterior, vmin = 0, vmax = 1)
            ax[0,0].set_aspect('auto')
            ax[0,0].set_title('P[Z=0|X] (prob not affected)')
            ax[0,0].set_xticks(np.arange(M))
            ax[0,0].set_xticklabels(names,rotation=15, fontsize = 5)
            ax[0,0].set_ylabel('Sample')

            ax[0,1].cla()
            ax[0,1].bar(np.arange(M),P)
            ax[0,1].set_xticks(np.arange(M))
            ax[0,1].set_xticklabels(names,rotation=15, fontsize = 5)
            ax[0,1].set_ylim((0, 1))

            f.canvas.draw()
    return P

## 5. Generate samples, parameter estimation, permutation testing

**The following code chunk was copied from the code to generate data** 

- It was copied from experiment 1 (clip the probabilities at 0.999 and 0.001, original), case 1 (nothing is affected) for 1,000 permutations.
- The data generated was in the general form: `case_1_experiment_1_repeat_xxxx.npz` for repeats 0000 to 0999

**Question:** How should the outputs be named/stored once I put this code in a general function?

In [ ]:
M = subset.shape[0] # Number of total unique structures
S = np.array(subset, dtype = bool)
names_subset = subset.columns # List of the 8 structures' names
n_repeats = 1000 # Number of repeats per case per experiment
nperm = 1000 # Number of permutations when permutation testing



for j in range(n_repeats):
    experiment_1_1_outputs = [] # Empty list for each iteration
    
    ### GENERATING SAMPLES ### 
    
    N = 20 # Number of samples
    mu = 3.0 # Difference in mean, note that this is generally unknown. 
    M = subset.shape[0] # Number of total unique structures
    number_of_leaves = np.count_nonzero(np.sum(subset, 1) == 0) # Number of leaf structures (zero children)

    Z = np.zeros((N,M)) # Initialize Z, which will be a binary variable that tells us if a structure is affected
    Naffected = N // 2 # Don't set Naffected to 0 or else there won't be any samples
            
    is_leaf = np.concatenate([np.ones(number_of_leaves), np.zeros(M - number_of_leaves)]) # 1 for leaf structures, 0 for non-leaf structures
    is_leaf = np.array(is_leaf, dtype = bool) # Convert is_leaf to the boolean type
    is_leaf = is_leaf[::-1] # Data specific
    m = np.sum(is_leaf) # Number of leaf structures (m = 2)
            
    G = np.arange(N) < Naffected # All falses since all samples are unaffected
    X = Z[:, is_leaf > 0] * mu + np.random.randn(N, m)
    
    ### PARAMETER ESTIMATION ###
    
    P_subset1_1 = np.ones(M) * 0.5 # Array of 8 copies of 0.5
    Q = Q_from_P(P_subset1_1, S)

    P0 = np.ones(M) * 0.5
    niter = 20
    P_subset1_1 = estimate_P(X[G], mu, S, Descendants_and_self, draw=0, P0=P0, niter=niter, names=names_subset)
    # Set draw = 0 to prevent drawing the graphs
    
    ### GENERATING PERMUTED DATA ###
    
    Ps = []
    for n in range(nperm):
        Xp = X[np.random.permutation(N)[G]]
        P_ = estimate_P(Xp,mu,S,Descendants_and_self,draw=0,niter=niter,P0=P0)
        Ps.append(P_)

    Ps_sort = np.array([np.sort(Pi)[::-1] for Pi in Ps])
    #Ps_sort = np.array(Ps)
    
    ### PERMUTATION TESTING ###
    
    inds = np.argsort(P_subset1_1)[::-1]
    pval = np.zeros_like(P_subset1_1)
    alpha = 0.05
    for i in range(M):    
    
        pval[inds[i]] = np.mean(Ps_sort[:,i] >= P_subset1_1[inds[i]])
        #if pval[inds[i]] > 0.05:
         #   break # Every structure that is not rejected
    
        experiment_1_1_outputs.append(f"{names_subset[inds[i]]}, P[Z=1|X]={P_subset1_1[inds[i]]}, p={pval[inds[i]]}") 
        # Every structure that gets rejected gets an entry
        
    file_name = f'case_1_experiment_1_repeat_{j:04}.npz' # format i so that it's 0 padded on the left with 4 characters
    np.savez(file_name, experiment_1_1_outputs)
    